In [13]:
import pandas as pd
import numpy as np


This file was originally for frequency distributions for Latin names!
but we can also concat the other dfs / do model training after in the same file

In [14]:
df_indo = pd.read_csv('df_indo.csv')
df_malay = pd.read_csv('df_malay.csv')
df_viet = pd.read_csv('viet_df.csv')
# also import other dfs

### 1. Cleaning up column names

In [17]:
# finding what columns r in viet that aren't in indo
print(df_indo.columns)
print(df_viet.columns)
[col for col in df_viet.columns if col not in df_indo.columns]

Index(['fullname', 'original_fullname', 'alphabet', 'unigrams', 'bigrams',
       'trigrams', 'char_ngrams', 'word_ngrams', 'name_length',
       'avg_token_length', 'num_tokens', 'transliteration', 'period_freq',
       'dash_freq', 'apostrophe_freq', 'space_freq', 'indiv_unigrams_fdist',
       'indiv_bigrams_fdist', 'indiv_trigrams_fdist', 'unigrams_cosine_sim',
       'bigrams_cosine_sim', 'trigrams_cosine_sim', 'label'],
      dtype='object')
Index(['fullname', 'alphabet', 'word_length', 'num_tokens', 'char_ngrams',
       'transliteration', 'unigrams', 'bigrams', 'trigrams',
       'indiv_unigrams_fdist', 'indiv_bigrams_fdist', 'unigrams_cosine_sim',
       'bigrams_cosine_sim'],
      dtype='object')


['word_length']

In [18]:
# idk if i can generalize this lol
# def rename_columns(df_ref, df_to_change):
#     diff_cols = [col for col in df_to_change.columns if col not in df_ref.columns]
#     new_names = 
#     df_to_change.rename(columns = {oldName : newName})

In [19]:
df_viet.rename(columns = {'word_length': 'name_length'}, inplace = True)
df_viet.head()

,fullname,alphabet,name_length,num_tokens,char_ngrams,transliteration,unigrams,bigrams,trigrams,indiv_unigrams_fdist,indiv_bigrams_fdist,unigrams_cosine_sim,bigrams_cosine_sim
0,từ hoàng thông,"['LATIN', 'LATIN', 'SPACE', 'LATIN', 'LATIN', ...",14,3,"[('t',), ('ừ',), (' ',), ('h',), ('o',), ('à',...",tu hoang thong,"['t', 'ừ', ' ', 'h', 'o', 'à', 'n', 'g', ' ', ...","[('t', 'ừ'), ('ừ', ' '), (' ', 'h'), ('h', 'o'...","[('t', 'ừ', ' '), ('ừ', ' ', 'h'), (' ', 'h', ...",[[0.14285714 0. 0. 0. ...,[[0. 0. 0. ... 0. 0. 0.]],0.805625,0.508198
1,nguyễn thị phương thảo,"['LATIN', 'LATIN', 'LATIN', 'LATIN', 'LATIN', ...",22,4,"[('n',), ('g',), ('u',), ('y',), ('ễ',), ('n',...",nguyen thi phuong thao,"['n', 'g', 'u', 'y', 'ễ', 'n', ' ', 't', 'h', ...","[('n', 'g'), ('g', 'u'), ('u', 'y'), ('y', 'ễ'...","[('n', 'g', 'u'), ('g', 'u', 'y'), ('u', 'y', ...",[[0.13636364 0. 0. 0. ...,[[0. 0. 0. ... 0. 0. 0.]],0.884792,0.667716
2,nick út,"['LATIN', 'LATIN', 'LATIN', 'LATIN', 'SPACE', ...",7,2,"[('n',), ('i',), ('c',), ('k',), (' ',), ('ú',...",nick ut,"['n', 'i', 'c', 'k', ' ', 'ú', 't']","[('n', 'i'), ('i', 'c'), ('c', 'k'), ('k', ' '...","[('n', 'i', 'c'), ('i', 'c', 'k'), ('c', 'k', ...",[[0.14285714 0. 0. 0.14285714 ...,[[0. 0. 0. ... 0. 0. 0.]],0.592690,0.005600
3,cao văn lầu,"['LATIN', 'LATIN', 'LATIN', 'SPACE', 'LATIN', ...",11,3,"[('c',), ('a',), ('o',), (' ',), ('v',), ('ă',...",cao van lau,"['c', 'a', 'o', ' ', 'v', 'ă', 'n', ' ', 'l', ...","[('c', 'a'), ('a', 'o'), ('o', ' '), (' ', 'v'...","[('c', 'a', 'o'), ('a', 'o', ' '), ('o', ' ', ...",[[0.18181818 0.09090909 0. 0.09090909 ...,[[0. 0. 0. ... 0. 0. 0.]],0.665965,0.243176
4,tạ thu thâu,"['LATIN', 'LATIN', 'SPACE', 'LATIN', 'LATIN', ...",11,3,"[('t',), ('ạ',), (' ',), ('t',), ('h',), ('u',...",ta thu thau,"['t', 'ạ', ' ', 't', 'h', 'u', ' ', 't', 'h', ...","[('t', 'ạ'), ('ạ', ' '), (' ', 't'), ('t', 'h'...","[('t', 'ạ', ' '), ('ạ', ' ', 't'), (' ', 't', ...",[[0.18181818 0. 0. 0. ...,[[0. 0. 0. ... 0. 0. 0.]],0.596114,0.288942


In [21]:
# note: viet dataset has no periods, dashes frequencies

### 4. Keeping numerical columns only for each dataset

In [27]:
# DO FOR EACH DATASET
# dropping non-numerical columns - this is why it would be good to have more numerical features since we don't have a lot
df_indo = df_indo.select_dtypes(exclude = 'object')
df_indo

,name_length,avg_token_length,num_tokens,period_freq,dash_freq,apostrophe_freq,space_freq,unigrams_cosine_sim,bigrams_cosine_sim
0,9,9.000000,1,0,0,0,0,0.664809,0.250640
1,12,12.000000,1,0,0,0,0,0.686625,0.353292
2,8,8.000000,1,0,0,0,0,0.688312,0.197139
3,9,9.000000,1,0,0,0,0,0.581396,0.155386
4,8,8.000000,1,0,0,0,0,0.463215,0.176917
...,...,...,...,...,...,...,...,...,...
11241,34,4.833333,6,0,0,0,5,0.883198,0.323446
11242,41,6.000000,6,0,0,0,5,0.906658,0.559409
11243,36,5.166667,6,4,0,0,5,0.731478,0.336306
11244,53,6.714286,7,0,0,0,6,0.905343,0.577418


In [23]:
# DROP FOR WHOLE DATASET
# dropping trigrams for indo and malay
df_indo.drop('trigrams_cosine_sim', inplace = True, axis = 1)
df_malay.drop('trigrams_cosine_sim', inplace = True, axis = 1)

In [24]:
# clean up viet / other dfs before combining

### 5. Adding the language (label) to each dataset

In [26]:
df_indo['language'] = 'Indonesian'
df_malay['language'] = 'Malay'
df_viet['language'] = 'Vietnamese'
df_indo.head()

,name_length,avg_token_length,num_tokens,period_freq,dash_freq,apostrophe_freq,space_freq,unigrams_cosine_sim,bigrams_cosine_sim,language
0,9,9.0,1,0,0,0,0,0.664809,0.250640,Indonesian
1,12,12.0,1,0,0,0,0,0.686625,0.353292,Indonesian
2,8,8.0,1,0,0,0,0,0.688312,0.197139,Indonesian
3,9,9.0,1,0,0,0,0,0.581396,0.155386,Indonesian
4,8,8.0,1,0,0,0,0,0.463215,0.176917,Indonesian


### 6. Combining all names to make one big dataset

In [ ]:
# as you can see from output, we need the columns in the concatenated df (in this case, viet) to match
# it's okay if some values are NaN bc we'll drop all non-numerical columns anyway
merged_df = pd.concat([df_indo, df_malay, df_viet], ignore_index = True, join = 'outer')
merged_df

### Steps

Next steps:
1. Clean up columns so we can combine dataframes into one (focus on making an all-Latin dataset first)
    - do not combine in this step
2. Frequency distributions for Latin names -> redo
3. Add a_hat_freq
4. Only keep numerical columns
    - turn some categorical features -> numerical so we have more things to feed into model
5. Add in label (language) for each dataset
6. Combine Latin and non-Latin names to make one big dataset
    - may need to repeat some of the above steps for non-Latin names
7. Train test split
8. MODEL TRAINING!
9. Model evaluation

Reminder:
- We decided to keep period_freq, dash_freq, apostrophe_freq for now. After our first run of model training, we can remove them to see if it improves the performance

**You can work on these steps out of order** (act as if the previous steps r there), but in the end we ideally want all of these steps implemented in this order.

For example, you could write the code for model training and train the model on one or a few datasets. Later on, we'll just replace the variables you used with the ones containing all the languages/names.